In [1]:
# Everything I know I learned from https://developers.google.com/optimization/mip/mip_var_array
# and this helped get ortools working https://stackoverflow.com/a/59998065/4144646

import pandas as pd
from ortools.linear_solver import pywraplp

# Men

In [9]:
# Create DataFrame
df = pd.read_csv('../data/FSA_DS_Men2020.csv')
df = df[['Rider Name', 'Price', 'Score 2019']] # using last year's score because ML didn't work out
df['slot'] = 1
df['24+'] = (df['Price'] >= 24).astype(int)
df['18+'] = (df['Price'] >= 18).astype(int)
df

,Rider Name,Price,Score 2019,slot,24+,18+
0,Primož Roglič,38,4070,1,1,1
1,Alejandro Valverde,28,2910,1,1,1
2,Egan Bernal,36,2752,1,1,1
3,Julian Alaphilippe,36,2601,1,1,1
4,Pascal Ackermann,28,2462,1,1,1
...,...,...,...,...,...,...
1237,Kamil Zielinski,1,0,1,0,0
1238,Maikel Zijlaard,1,0,1,0,0
1239,Georg Zimmerman,1,0,1,0,0
1240,Nickolas Zukowsky,1,0,1,0,0


In [10]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['constraint_coeffs'] = [list(df['Price']),
                                 list(df['slot']),
                                 list(df['24+']),
                                 list(df['18+'])]
    data['bounds'] = [150, 25, 1, 3]
    data['obj_coeffs'] = list(df['Score 2019'])
    data['num_vars'] = len(df)
    data['num_constraints'] = len(data['bounds'])
    return data

data = create_data_model()
# Create the mip solver with the CBC backend.
solver = pywraplp.Solver('simple_mip_program', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
infinity = solver.infinity()
x = {}
for j in range(data['num_vars']):
    x[j] = solver.IntVar(0, 1, df.loc[j, 'Rider Name'])
print('Number of variables =', solver.NumVariables())

for i in range(data['num_constraints']):
    constraint = solver.RowConstraint(0, data['bounds'][i], '')
    for j in range(data['num_vars']):
        constraint.SetCoefficient(x[j], data['constraint_coeffs'][i][j])
print('Number of constraints =', solver.NumConstraints())
# In Python, you can also set the constraints as follows.
# for i in range(data['num_constraints']):
#    constraint_expr = \
# [data['constraint_coeffs'][i][j] * x[j] for j in range(data['num_vars'])]
#    solver.Add(sum(constraint_expr) <= data['bounds'][i])

objective = solver.Objective()
for j in range(data['num_vars']):
    objective.SetCoefficient(x[j], data['obj_coeffs'][j])
objective.SetMaximization()
# In Python, you can also set the objective as follows.
# obj_expr = [data['obj_coeffs'][j] * x[j] for j in range(data['num_vars'])]
# solver.Maximize(solver.Sum(obj_expr))

status = solver.Solve()

selected_riders = []

if status == pywraplp.Solver.OPTIMAL:
    print('Objective value =', solver.Objective().Value())
    for j in range(data['num_vars']):
        if x[j].solution_value():
            selected_riders.append(j)
    print()
    print('Problem solved in %f milliseconds' % solver.wall_time())
    print('Problem solved in %d iterations' % solver.iterations())
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
else:
    print('The problem does not have an optimal solution.')
    
df.loc[selected_riders]

Number of variables = 1242
Number of constraints = 4
Objective value = 18117.0

Problem solved in 171.000000 milliseconds
Problem solved in 0 iterations
Problem solved in 0 branch-and-bound nodes


,Rider Name,Price,Score 2019,slot,24+,18+
0,Primož Roglič,38,4070,1,1,1
13,Bauke Mollema,16,1690,1,0,0
20,Mike Teunissen,14,1507,1,0,0
26,Diego Ulissi,14,1400,1,0,0
48,Amund Groendahl Jansen,8,912,1,0,0
52,Jack Haig,8,875,1,0,0
71,Guillaume Martin,6,695,1,0,0
72,Andrea Vendrame,6,685,1,0,0
73,Benoit Cosnefroy,6,675,1,0,0
102,Ivan Garcia Cortina,4,495,1,0,0


In [11]:
df.loc[selected_riders].sum()

Rider Name    Primož RogličBauke MollemaMike TeunissenDiego ...
Price                                                       150
Score 2019                                                18117
slot                                                         25
24+                                                           1
18+                                                           1
dtype: object

# Women

In [12]:
# Create DataFrame
df = pd.read_csv('../data/FSA_DS_Women2020.csv')
df = df[['Rider Name', 'Price', 'Score 2019']] # using last year's score because ML didn't work out
df['slot'] = 1
df['20+'] = df['Price'].apply(lambda x: x if x >= 20 else 0)
df

,Rider Name,Price,Score 2019,slot,20+
0,Annemiek van Vleuten,52,4260,1,52
1,Marianne Vos,50,4020,1,50
2,Lorena Wiebes,46,3373,1,46
3,Marta Bastianelli,42,3348,1,42
4,Anna van der Breggen,52,2956,1,52
...,...,...,...,...,...
578,Sophie Wright,1,0,1,0
579,Fernanda Yapura,1,0,1,0
580,Silvia Zanardi,1,0,1,0
581,Emanuela Zanetti,1,0,1,0


In [13]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['constraint_coeffs'] = [list(df['Price']),
                                 list(df['slot']),
                                 list(df['20+'])]
    data['bounds'] = [150, 15, 100]
    data['obj_coeffs'] = list(df['Score 2019'])
    data['num_vars'] = len(df)
    data['num_constraints'] = len(data['bounds'])
    return data

data = create_data_model()
# Create the mip solver with the CBC backend.
solver = pywraplp.Solver('simple_mip_program', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
infinity = solver.infinity()
x = {}
for j in range(data['num_vars']):
    x[j] = solver.IntVar(0, 1, df.loc[j, 'Rider Name'])
print('Number of variables =', solver.NumVariables())

for i in range(data['num_constraints']):
    constraint = solver.RowConstraint(0, data['bounds'][i], '')
    for j in range(data['num_vars']):
        constraint.SetCoefficient(x[j], data['constraint_coeffs'][i][j])
print('Number of constraints =', solver.NumConstraints())
# In Python, you can also set the constraints as follows.
# for i in range(data['num_constraints']):
#    constraint_expr = \
# [data['constraint_coeffs'][i][j] * x[j] for j in range(data['num_vars'])]
#    solver.Add(sum(constraint_expr) <= data['bounds'][i])

objective = solver.Objective()
for j in range(data['num_vars']):
    objective.SetCoefficient(x[j], data['obj_coeffs'][j])
objective.SetMaximization()
# In Python, you can also set the objective as follows.
# obj_expr = [data['obj_coeffs'][j] * x[j] for j in range(data['num_vars'])]
# solver.Maximize(solver.Sum(obj_expr))

status = solver.Solve()

selected_riders = []

if status == pywraplp.Solver.OPTIMAL:
    print('Objective value =', solver.Objective().Value())
    for j in range(data['num_vars']):
        if x[j].solution_value():
            selected_riders.append(j)
    print()
    print('Problem solved in %f milliseconds' % solver.wall_time())
    print('Problem solved in %d iterations' % solver.iterations())
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
else:
    print('The problem does not have an optimal solution.')
    
df.loc[selected_riders]

Number of variables = 583
Number of constraints = 3
Objective value = 13526.0

Problem solved in 81.000000 milliseconds
Problem solved in 11 iterations
Problem solved in 0 branch-and-bound nodes


,Rider Name,Price,Score 2019,slot,20+
0,Annemiek van Vleuten,52,4260,1,52
7,Soraya Paladin,24,2141,1,24
20,Sofie De Vuyst,14,1290,1,0
26,Elena Cecchini,12,1041,1,0
35,Leah Thomas,8,715,1,0
37,Vita Heine,8,705,1,0
38,Alison Jackson,8,696,1,0
45,Ane Santesteban,6,580,1,0
59,Alena Amialyusik,4,421,1,0
60,Kathrin Hammes,4,419,1,0


In [14]:
df.loc[selected_riders].sum()

Rider Name    Annemiek van VleutenSoraya PaladinSofie De Vuy...
Price                                                       150
Score 2019                                                13526
slot                                                         15
20+                                                          76
dtype: object